<h1 style="background-color: green;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐תהליך הסבת גבולות מפות הסדר לבנק"ל מודרני🌐

In [ ]:
print('10. Blocks regulation borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import dask_geopandas as daskgpd
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  10.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר קיימת

In [ ]:
CadasterProcessBorder_cols = ['CADASTER_PROCESS', 'GUSHNUM', 'GUSHSUFFIX', 'GUSH_STATUS', 'geometry']

CadasterProcessBorder = daskgpd.read_file(CNFG.CADSDE + 'CadasterProcessBorder.gdb', layer = 'CadasterProcessBorder', columns = CadasterProcessBorder_cols, npartitions = 16).compute()

del CadasterProcessBorder_cols

<h1 style="text-align:right;font-size:100%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

<h1 style="text-align:right;font-size:100%">
סינון גבולות תהליכי קדסטר קיימים לפי גבולות הסדר גוש

In [ ]:
print('  10.b Cleaning data')

In [ ]:
regulation_borders = CadasterProcessBorder[CadasterProcessBorder['CADASTER_PROCESS'] == 1]
regulation_borders = regulation_borders.drop(columns = 'CADASTER_PROCESS')

del CadasterProcessBorder

In [ ]:
print('  10.c Data conversion')

<h1 style="text-align:right;font-size:100%">
מתן שם מפה לתהליך

In [ ]:
regulation_borders['GUSHSUFFIX'] = regulation_borders['GUSHSUFFIX'].fillna(0)

regulation_borders[['GUSHNUM', 'GUSHSUFFIX']] = regulation_borders[['GUSHNUM', 'GUSHSUFFIX']].astype(int)

regulation_borders['ProcessName'] = regulation_borders['GUSHNUM'].astype(str) + '/' + regulation_borders['GUSHSUFFIX'].astype(str) + ' הסדר'

<h1 style="text-align:right;font-size:100%">
יצירת שדה משותף של מספר גוש מלא ואיחוד עם שכבת גושי קדסטר מודרנית לקבלת מזהה גוש

In [ ]:
regulation_borders['BlockName'] = regulation_borders['GUSHNUM'].astype(str) + '/' + regulation_borders['GUSHSUFFIX'].astype(str)

regulation_borders = regulation_borders.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')

del Blocks

<h1 style="text-align:right;font-size:100%">
הסרת כפילויות

In [ ]:
regulation_borders = regulation_borders.drop_duplicates(subset = 'BlockName', keep=False)

In [ ]:
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].astype(int)

status_clf = {0    : 90,
              7    : 90,
              9    : 90,
              10   : 90,
              11   : 91,
              13   : 95,
              15   : 99,
              17   : 90,
              None : 90}


regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].fillna(0)
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].map(status_clf)
regulation_borders['GUSH_STATUS'] = regulation_borders['GUSH_STATUS'].astype(int)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  10.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ProcessType", int),
                        #("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),
                        ("Status", int),
                        #("PlanName", str),
                        #("SurveyorLicenseID", int),
                        ("DataSource", int),
                        ("geometry", str)
                        ])


BlocksRegulationBorders = gpd.GeoDataFrame(np.empty(0, dtype=new_columns),
                                           geometry = 'geometry',
                                           crs = 2039)

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
BlocksRegulationBorders['geometry'] = regulation_borders['geometry']

BlocksRegulationBorders['DataSource'] = 5 #CAD

#BlocksRegulationBorders['SurveyorLicenseID'] = 0   # לא קיים מידע

#BlocksRegulationBorders['PlanName'] = None   # לא קיים מידע

BlocksRegulationBorders['Status'] = regulation_borders['GUSH_STATUS'].astype(int)

BlocksRegulationBorders['GeodeticNetwork'] = 3   # רשת ישראל התקפה

BlocksRegulationBorders['ProcessName'] = regulation_borders['ProcessName']

#BlocksRegulationBorders['MapYear'] = None

BlocksRegulationBorders['ProcessType'] = 9   # הסדר מקרקעין

BlocksRegulationBorders['BlockUniqueID'] = regulation_borders['BlockUniqueID'].astype('Int64')

BlocksRegulationBorders['CPBUniqueID'] = range(1, len(BlocksRegulationBorders)+1)
BlocksRegulationBorders['CPBUniqueID'] = BlocksRegulationBorders['CPBUniqueID'] + 9000000
BlocksRegulationBorders['CPBUniqueID'] = BlocksRegulationBorders['CPBUniqueID'].astype(int)

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  10.e Exporting results to product folder')

In [ ]:
json_dir =  CNFG.JsonPath + 'BlocksRegulationBorders.geojson'

BlocksRegulationBorders.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'BlocksRegulationBorders-Save as Feature Class.bat']);